# IMT574 Problem Set 7: 

In [1]:
# Importing required libraries and pacakages
import numpy as np
import pandas as pd
import random
from statistics import mean
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, recall_score
from sklearn.preprocessing import normalize
import warnings

#### 1.1 

In [2]:
# Importing data
r_df = pd.read_csv('./amazon-reviews.csv.bz2', sep='\t')
r_df.head(5)

,date,summary,review,rating
0,2013-07-16,Awesine,Perfect for new parents. We were able to keep ...,5
1,2013-06-29,Should be required for all new parents!,This book is such a life saver. It has been s...,5
2,2014-03-19,Grandmother watching baby,Helps me know exactly how my babies day has go...,5
3,2013-08-17,repeat buyer,I bought this a few times for my older son and...,5
4,2014-04-01,Great,I wanted an alternative to printing out daily ...,4


In [3]:
r_df.shape

(205331, 4)

In [4]:
# Number of nulls in reviews column
r_df.review.isna().sum()

80

In [5]:
# Dropping null values
r_dfc = r_df.dropna(subset=['review'])

In [6]:
r_dfc.shape

(205251, 4)

In [7]:
# Checking for blanks in reviews column 
filt = r_dfc["review"] != ""
r_dfc = r_dfc[filt]

In [8]:
r_dfc.shape

(205251, 4)

##### 1.2

In [9]:
# Setting a random seed to generate a sample of 1000 reviews
np.random.seed(123)
# Taking a random sample of 1000 reviews
r_sample = r_dfc.sample(n=1000)

In [10]:
r_sample.head()

,date,summary,review,rating
141133,2012-08-03,works as it should,We bought this to go with our First Year Ignit...,4
95579,2013-01-09,Not Good for You or Your Baby for Extended Per...,~BACKGROUND~I purchased this to use for my new...,3
116319,2013-08-25,fits perfect,fits my 14 month old perfectly...washes up gre...,5
175444,2016-12-03,love,"Purfect coverage, love it",5
83080,2013-07-26,Great reasonable lightweight stroller,we have the uppa baby vista stroller which is ...,4


##### 1.3

In [11]:
# Inspecting reviews
pd.options.display.max_colwidth = 1000
r_sample.review.tail(10)

199800                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              I bought this stand because I needed support for larger songbooks that my metal fold-up could not provide.  I really, really like it. Others in my music groups that have seen it have also gone out and ordered one.  It assembles quickly and easily, and folds up into a nice package for transport.  Every once in a while I have to tighten the lock cams, but that is no issue.  It is very sturdy and holds big music books v

## 2 TF-IDF

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
#Count Vectorizer for reviews
vr = CountVectorizer()
X = vr.fit_transform(r_sample.review.values).toarray()
df = pd.DataFrame(X, columns = vr.get_feature_names())
tf = np.log(1+df)
denom = (1+(df>0).sum(axis=0))
idf = np.log(X.shape[0]/denom)
tf_idf = tf*idf
tf_idf.head()

,00,01,04,07,08,09,10,100,10lbs,10p,...,zigzag,zinc,zip,ziplocs,zipper,zippers,zips,zoo,zoom,zs
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.346732,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3 k-Means Clustering

In [28]:
tf_idf.head(5)

,00,01,04,07,08,09,10,100,10lbs,10p,...,zigzag,zinc,zip,ziplocs,zipper,zippers,zips,zoo,zoom,zs
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.346732,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
def km(k, tf_idf):
    
    # Setting random seed for reproducibility
    np.random.seed(1)
    # Setting k random cluster centres
    centres = tf_idf.sample(n=k)
    
    # Creating a list of indexes of cluster centres
    centre_ind = centres.index.tolist()
    
    # Centroids for  the first iteration
    centroids = []
    for i in range(len(centre_ind)):
        cluster = []
        cluster.append(centre_ind[i])
        centroids.append(tf_idf.iloc[cluster].mean(axis=0).tolist())

    # Creating a dataframe of these centroids 
    centroid_df = pd.DataFrame(centroids)

    # Calculating the cos similarities between all reviews and cluster centroids
    cos = []
    for i in range(k): # Iterating over each cluster
        cos_in = []
        for j in range(tf_idf.shape[0]): # Iterating over every review
            dot = np.dot(tf_idf.iloc[j],centroid_df.iloc[i]) # Calculating dot product between review and centroid
            anorm = np.linalg.norm(tf_idf.iloc[j]) # Calculating norm of review
            bnorm = np.linalg.norm(centroid_df.iloc[i]) # Calculating norm of centroid
            cos_sim = dot / (anorm * bnorm) # Cosine similarity between review and centroid
            cos_in.append(cos_sim)
        cos.append(cos_in)
    cos_df = (pd.DataFrame(cos)).transpose()
    
    # Assigning review to cluster which has the least cosine similarity value
    cos_df['cluster'] = cos_df.idxmin(axis=1)+1
    cos_df.head()
    
    listclust = cos_df.cluster.unique()
    
    # Further iterations of k-means clustering

    # Initializing a dataframe to hold new centroids
    newcentroid_df = centroid_df.copy()

    # Initializing an empty dataframe to store older centroids
    prevcentroid_df = pd.DataFrame(columns=centroid_df.columns)
    prevcentroid_df = prevcentroid_df.fillna(0)

    # Intializing a dataframe for holding new cosine similarity values for all rows and cluster centres
    newcos_df = cos_df.copy()

    # Counter for number of iterations
    iteration_no = 2

    # Continuing to iterate till previous centroids match new centroids and number of iterations is less than 30
    while ((prevcentroid_df.equals(newcentroid_df) == False) & (iteration_no < 35)):

        #print(iteration_no)

        prevcentroid_df = newcentroid_df.copy()  # centroids from previous iteration

        newcentroidlist = []
        for i in range(len(listclust)):
            sublist = newcos_df[newcos_df.cluster == listclust[i]].index.tolist()
            newcentroidlist.append(sublist)

        newcentroids = []

        # Computing new centroids for each cluster by averaging the tf_idf columns
        for cluster in newcentroidlist:
            centroid_list = tf_idf.iloc[cluster].mean(axis=0).tolist()
            newcentroids.append(centroid_list)

        # New Centroids
        newcentroid_df = pd.DataFrame(newcentroids) # centroids from current iteration

        newcos = []
        # Calculating the cos similarities between all reviews and cluster centroids
        for i in range(k):
            cos_in = []
            for j in range(tf_idf.shape[0]):
                dot = np.dot(tf_idf.iloc[j],newcentroid_df.iloc[i])
                anorm = np.linalg.norm(tf_idf.iloc[j])
                bnorm = np.linalg.norm(newcentroid_df.iloc[i])
                cos_sim = dot / (anorm * bnorm)
                cos_in.append(cos_sim)
            newcos.append(cos_in)
        newcos_df = (pd.DataFrame(newcos)).transpose()
        newcos_df['cluster'] = newcos_df.idxmin(axis=1)+1
        iteration_no = iteration_no + 1

    return newcos_df

In [91]:
final = km(3,tf_idf)

What kind of properties of the reviews is the algorithm considering? What constitutes similar
reviews?

In [92]:
final.cluster.value_counts()

2    933
1     36
3     31
Name: cluster, dtype: int64

In [103]:
listclust = final.cluster.unique().tolist()
listclust

[2, 3, 1]

In [104]:
finalclusters = []
for i in range(len(listclust)):
    sublist = final[final.cluster == listclust[i]].index.tolist()
    finalclusters.append(sublist)

It looks like the algorithm is looking at words used in the reviews to classify them into clusters. For example, the review "I really like this makeup. It blends in well, it's nice and airy, not heavy or greasy. This color compliments my summer skin tone nicely, a light tan with neutral undertones. I can wear this and be confident that I already have sunscreen protection during the day. It is matte, but I can add shimmer powder or blush on top if I want. I like the coverage and the tint, so I would order more." has been classified correctly in newcluster3 along with other makeup reviews. 

Similar reviews are clustered based on the cosine similarity which is how similar two reviews are in terms of word count of "key" words. For instance, "Good polish" and "Go to polish" are both classified as reviews from the beauty section. This is probably because both reviews have the word "polish". 

### 4

#### 4.1 

In [84]:
# Resetting index of 1000 observation sample
reset_sample = r_sample.reset_index()
reset_sample.head()

,index,date,summary,review,rating
0,141133,2012-08-03,works as it should,"We bought this to go with our First Year Ignite stroller and it works just fine. I like how it folds up for easy storage and it goes on quite easily. I guess the only thing I have to complain about it (and yes I realize this is petty) is that it looks like a witches hat LOL. I find it rather silly looking when viewing it from above. But honestly, it works great and does the job (covers up my baby in his stroller) so what more can you ask for other than a different color maybe?",4
1,95579,2013-01-09,Not Good for You or Your Baby for Extended Periods of Time,"~BACKGROUND~I purchased this to use for my newborn son. I am ~6'2"" and 175 lbs and my wife is 5'4"" and we have both used the carrier.~PROS~- THE PRICE!- Appears to be made of good materials and is of good workmanship.- Padded straps.- Quick release buckles make it fairly easy to put on and open and close the carrier.- Detachable bib.- Gender neutral and father friendly design/print/colors.- Face-in and face-out carrying positions.- Soft and comfortable.- Compact size.- Open sides allow your child to grow into the carrier.- Fabric is washable/wipeable.- Adjustable ""one-touch"" back strap.~CONS~- All of the baby's weight is on his/her crotch.- All of the weight of the baby is on your shoulders.- The description states 8-25 lbs, but I think 8 lbs is a bit too small for this carrier as there are buckles and straps right at face level.- Does not offer the most ideal support for smaller babies as they tend to roll around inside the carrier.- Can be difficult to put a child in or take out...",3
2,116319,2013-08-25,fits perfect,"fits my 14 month old perfectly...washes up great, nice to not use 5 bibs a day and great to keep her arms clean too",5
3,175444,2016-12-03,love,"Purfect coverage, love it",5
4,83080,2013-07-26,Great reasonable lightweight stroller,we have the uppa baby vista stroller which is awesome but it's bulky. we wanted to find something that was easy to use in smaller spaces and easy to carry (i'm only 5'2&#34; with a growing boy)! this works just fine; i find that it's better quality than most in this price range. don't expect super high end like some of the ones you'll spend a lot more on but for the price i think it offers a lot.pros: the seat adjusts forward and back which is great for our son. has a harness clasp which is safer. there is a canopy which keeps most (not all) of the sun out and there are brakes on the wheels. the top handles have a cup holder and a small zip compartment. we haven't been in a situation where it's been totally soiled but it's been very easy to clean so far.we're very happy with this and i'd recommend if you're looking for an easy to use lightweight stroller that doesn't cost a fortune.,4


#### 4.2

In [85]:
reset_sample['review'].iloc[finalclusters[0]]

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            We bought this to go with our First Year Ignite stroller and it works just fine. I like how it folds up for easy storage and it goes on quite easily. I guess the only thing I have to complain about it (and yes I realize this is petty) is that it looks like a witches hat LOL. I find it rather silly looking when viewing it from above. But honestly, it works great and does the job (covers up my baby in his stroller) so what more can you ask for other than a different color 

In [86]:
reset_sample['review'].iloc[finalclusters[1]]

56                                                                                                                                                                                                                                                                                                                           Love the monitor. It has good picture quality and gives us peace of mind. The base can be rotated to view anywhere in crib. It has a microphone to talk to baby and you can play music.
68                                                                                                                                                                                                                                                                                                                                                                                     Baby dearly loves this! I got a second in blue to be sure we had enough! The small horse is perfect for little hands to grab

In [88]:
reset_sample['review'].iloc[finalclusters[2]]

3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               Purfect coverage, love it
41                                                                                                                                            

#### 4.3

The algorithm with k=3 does not seem to cluster similar reviews together very accurately. We can see atleast one cluster (newcluster2) that does not seem to have any particular type of reviews. This is because the algorithm just checks for the word count and not the context. For instance, The following 2 are misclassified reviews:

"Love the monitor. It has good picture quality and gives us peace of mind. The base can be rotated to view anywhere in crib. It has a microphone to talk to baby and you can play music." - This review didn't get clustered into the baby review cluster (cluster1) because of the presence of words like "music", "monitor", "microphone"

"My son adores the flashing lights and the music.  I just wish it would play continuously because he isn't old enough to push the button himself.  But for playtime it is perfect and always captures his attention." - Similarly, this review has words like "lights","music", which may have caused it to get clustered incorrectly.


#### 4.4

In [99]:
final4 = km(4,tf_idf)
final4.cluster.value_counts()

1    655
2    337
4      8
Name: cluster, dtype: int64

In [100]:
listclust4 = final4.cluster.unique().tolist()

In [95]:
finalclusters = []
for i in range(len(listclust)):
    sublist = final4[final4.cluster == listclust4[i]].index.tolist()
    finalclusters.append(sublist)

In [97]:
final5 = km(5,tf_idf)
final5.cluster.value_counts()

3    987
4     10
1      3
Name: cluster, dtype: int64

In [101]:
final2 = km(2,tf_idf)
final2.cluster.value_counts()

2    644
1    356
Name: cluster, dtype: int64

#### 4.5

Even after taking different values of k like 2, 3, 4, 5 we realize that the ideal number of clusters for this data is 3 (since for k = 4 and k = 5 we lost 1 and 2 clusters, respectively and ended up having 3 clusters itself).

3 clusters gave us relatively distinct clusters with lesser overlap. k = 4 and k = 5 were interesting because we could see how the number of clusters still converged to 3 and did not remain 4 or 5.

We can say for k = 3, the results were fairly good based on the inspection of reviews in each clusters. 